In [ ]:
from openagentkit.modules.openai import OpenAIExecutor
from openagentkit.core.handlers.tools.tool_wrapper import tool
from typing import Annotated
import openai
import os

In [3]:
# Define a tool
@tool(
    description="Get the weather of a city", # Define the tool description
)
def get_weather(city: Annotated[str, "The city to get the weather of"]): # Each argument must be of type Annotated
    """Get the weather of a city"""

    # Actual implementation here...
    # ...

    return f"Weather in {city}: sunny, 20°C, feels like 22°C, humidity: 50%"

In [4]:
get_weather.schema

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get the weather of a city',
  'strict': True,
  'parameters': {'properties': {'city': {'title': 'City', 'type': 'string'}},
   'required': ['city'],
   'type': 'object',
   'additionalProperties': False}}}

In [5]:
@tool(
    description="Calculate the sum of 2 numbers", # Define the tool description
)
def calculate(a: Annotated[int, "a"], b: Annotated[int, "b"]) -> int:
    """Calculate the sum of two numbers"""
    return a + b

In [6]:
calculate.schema

{'type': 'function',
 'function': {'name': 'calculate',
  'description': 'Calculate the sum of two numbers',
  'strict': True,
  'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
    'b': {'title': 'B', 'type': 'integer'}},
   'required': ['a', 'b'],
   'type': 'object',
   'additionalProperties': False}}}

In [7]:
# Initialize OpenAI client
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Initialize LLM service
executor = OpenAIExecutor(
    client=client,
    model="gpt-4o-mini",
    system_message="""
    You are a helpful assistant that can answer questions and help with tasks.
    You are also able to use tools to get information.
    """,
    tools=[get_weather],
    temperature=0.5,
    max_tokens=100,
    top_p=1.0,
)

generator = executor.execute(
    messages=[
        {"role": "user", "content": "What's the weather like in New York?"}
    ]
)

for response in generator:
    print(response)

role='assistant' content=None tool_calls=[{'id': 'call_LmjgeXAZlZwLTTlmDjjoNUT7', 'type': 'function', 'function': {'arguments': '{"city":"New York"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=106, completion_tokens=16, total_tokens=122, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
role='tool' content=None tool_calls=None tool_results=[ToolCallResult(tool_name='get_weather', result='Weather in New York: sunny, 20°C, feels like 22°C, humidity: 50%')] refusal=None audio=None usage=None
role='assistant' content='The weather in New York is sunny with a temperature of 20°C (feels like 22°C) and humidity at 50%.' tool_calls=None tool_results=[] refusal=None audio=None usage=UsageResponse(prompt_tokens=156, completion_tokens=30, total_tokens=186, prompt_tokens

In [8]:
executor.get_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-05-03 00:00:28\n\n        \n        '},
 {'role': 'user', 'content': "What's the weather like in New York?"},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_LmjgeXAZlZwLTTlmDjjoNUT7',
    'type': 'function',
    'function': {'arguments': '{"city":"New York"}', 'name': 'get_weather'}}],
  'content': 'None'},
 {'role': 'tool',
  'tool_call_id': 'call_LmjgeXAZlZwLTTlmDjjoNUT7',
  'content': 'Weather in New York: sunny, 20°C, feels like 22°C, humidity: 50%'},
 {'role': 'assistant',
  'content': 'The weather in New York is sunny with a temperature of 20°C (feels like 22°C) and humidity at 50%.'}]

In [9]:
executor.clear_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-05-03 00:00:28\n\n        \n        '}]

In [10]:
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")
    
    if user_input.lower() == "exit":
        break

    generator = executor.execute(
        messages=[
            {"role": "user", "content": user_input}
        ]
    )

    for response in generator:
        if response.content is not None:
            print(f"AI: {response.content}")
        if response.tool_calls:
            [print(f"Tool Calls: {tool_call}") for tool_call in response.tool_calls]

User: exit


In [11]:
executor.get_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-05-03 00:00:28\n\n        \n        '}]